# Enhanced Skin Cancer Detection Using Diffusion-Based Data Augmentation and Vision Transformers

# Mounting the Drive

In [ ]:
# Mount the gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change path
%cd /content/drive/MyDrive/CS \198/Datasets/ISIC2016

/content/drive/.shortcut-targets-by-id/1asTaHB00qgGULpAu4P1_uXreHPv8cwux/CS 198/Datasets/ISIC2016


In [ ]:
%ls

ISBI2016_ISIC_Part1_Training_Data/  ISBI2016_ISIC_Part3_Training_GroundTruth.csv


# Importing Libraries

In [ ]:
import cv2

# Data Preprocessing

import dataset
resize images etc

# Diffusion-Based Augmentation

# Classification using Vision Transformers